In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_deterministic_ops=true"
import jax, json, matplotlib as mpl
from jax import numpy as jnp
from flax import nnx
from models import LeNet
from fedflax import train, aggregate, get_updates, cast
from data import fetch_data
from functools import reduce, partial
from matplotlib import pyplot as plt
from utils import return_l2, angle_err, opt_create, functional_drift
from collections import defaultdict
n_clients = 4

## Setup

In [ ]:
# Data 
ds_train = fetch_data(beta=1., skew="feature", n_clients=n_clients)
ds_val = fetch_data(partition="val", beta=1., batch_size=32, skew="feature", n_clients=n_clients)
ds_test = fetch_data(partition="test", beta=1., batch_size=16, skew="feature", n_clients=n_clients)

# Various settings to find conditions in which the asymmetry helps
hypers_sweep:tuple[dict,...] = (
    {"wasym":"densest", "kappa":1.},
    {"sigma":1e-4},
    {"orderbias":True},
    {"normweights":True},
    {"wasym":"random", "kappa":1.},
    {"sigma":1e-4, "weight_decay":2e-3},
    {"wasym":"densest", "kappa":1e-2},
    {"sigma":1e-3},
    {"wasym":"random", "kappa":1e-2},
    {"sigma":1e-3, "weight_decay":2e-3}
)

## Visualize

In [3]:
# Rendering style
plt.style.use("seaborn-v0_8-pastel")
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "font.sans-serif": ["Helvetica"],
    "text.latex.preamble": r"""
        \usepackage{amsmath, amssymb}
        \usepackage{mathptmx} 
    """
})

def plot_as_grid(data:dict[int:dict[int:float]], filename:str):
    fig, ax = plt.subplots(dpi=500);
    # Determine grid size
    y_ticks = list(data.keys())
    x_ticks = max([set(subdict.keys()) for subdict in data.values()], key=len)
    x_ticks = sorted(x_ticks.union({"0"}), key=int) # in case longest is nreg0
    grid = jnp.full((len(y_ticks), len(x_ticks)), jnp.nan)
    # Populate grid
    for i, n_regular_rounds in enumerate(y_ticks):
        for j, n_asym_rounds in enumerate(x_ticks):
            # Set value, or NaN if missing
            value = data[n_regular_rounds].get(n_asym_rounds, jnp.nan)
            grid = grid.at[i, j].set(value)
            # Simultaneously, plot the value as text
            text = f"{value:.2f}" if not jnp.isnan(value) else ""
            ax.text(j, i, text, ha="center", va="center", color="w");

    # Plot grid
    cmap = plt.get_cmap("inferno").with_extremes(bad="lightgray");
    ax.imshow(grid, cmap=cmap, norm=mpl.colors.LogNorm(vmin=3.66, vmax=15.5), interpolation="nearest");
    ax.set_xticks(range(len(x_ticks)), labels=[str(x) for x in x_ticks]);
    ax.set_yticks(range(len(y_ticks)), labels=[str(y) for y in y_ticks]);
    ax.set_ylabel("No. of conventional communication rounds performed in advance");
    ax.set_xlabel("No. of communication rounds subsequently\nperformed with asymmetry applied");
    fig.savefig(filename.removesuffix(".png")+".png", bbox_inches="tight");
    return fig

## Check metrics
Available techniques are dimension expansion and W-Asymmetry, of which the latter is implemented here.

In [4]:
for hypers in hypers_sweep:
    # Track metrics
    hypers_as_str = str(hypers)
    wd = hypers.pop("weight_decay", 1e-4)
    test_errs_global = defaultdict(dict)
    test_errs_local = defaultdict(dict)
    drift = defaultdict(dict)
    # Initialize regular model
    reg_model = LeNet(jax.random.key(42))
    # Number of rounds is initialized to number required for early stopping
    regular_roundses = jnp.array([(i+1)//2 for i in range(11)]).tolist()
    for j in range(len(regular_roundses)):
        # Federated training using regular model
        opt_create = partial(opt_create, learning_rate=1e-3, weight_decay=wd)
        if regular_roundses[j]:
            reg_models, _ = train(reg_model, opt_create, ds_train, return_l2(0.), ds_val, local_epochs="early", rounds=regular_roundses[j], val_fn=angle_err, max_patience=2, n_clients=n_clients)
            reg_model = aggregate(reg_model, get_updates(reg_model, reg_models))
        # Finalization of training using asymmetric model
        asym_roundses = jnp.array([(i+1)//2 for i in range(11)]).tolist()
        for i in range(len(asym_roundses)):
            if i==1:
                # Transfer parameters of the trained regular model to an asymmetric architecture
                reg_params = nnx.state(reg_model, (nnx.Param, nnx.BatchStat))
                asym_struct, _, rest = nnx.split(LeNet(jax.random.key(42), **hypers), (nnx.Param, nnx.BatchStat), ...) # key is irrelevant because params are overwritten
                asym_model = nnx.merge(asym_struct, reg_params, rest)
            if i==0:
                asym_model = reg_model
                asym_models = cast(reg_model, n_clients) if j==0 else reg_models
            else:
                # Train asymmetric
                asym_models, _ = train(asym_model, opt_create, ds_train, return_l2(0.), ds_val, local_epochs="early", rounds=asym_roundses[i], val_fn=angle_err, max_patience=2, n_clients=n_clients)
                # Aggregate final models (the model passed as first arguments is technically irrelevant)
                updates = get_updates(asym_model, asym_models)
                asym_model = aggregate(asym_model, updates)                
            # Accuracy on each client's data of aggregated model
            vval_fn = nnx.jit(nnx.vmap(angle_err, in_axes=(None,0,0,0)))
            err_test_global = reduce(lambda e, batch: e + vval_fn(asym_model, *batch), ds_test, 0.) / len(ds_test)
            test_errs_global[sum(regular_roundses[:j+1])][sum(asym_roundses[:i+1])] = err_test_global.mean().item()
            # Accuracy of local models
            vval_fn = nnx.jit(nnx.vmap(angle_err))
            err_test_local = reduce(lambda e, batch: e + vval_fn(asym_models, *batch), ds_test, 0.) / len(ds_test)
            test_errs_local[sum(regular_roundses[:j+1])][sum(asym_roundses[:i+1])] = err_test_local.mean().item()
            # Client drift measured in function space # TODO: doesn't make sense when training for multiple rounds
            drift[sum(regular_roundses[:j+1])][sum(asym_roundses[:i+1])] = functional_drift(asym_models, ds_test).mean().item()
            # Save intermediate results
            json.dump(test_errs_global, open(f"break/test_errs_global_{hypers_as_str}.json", "w"))
            json.dump(test_errs_local, open(f"break/test_errs_local_{hypers_as_str}.json", "w"))
            json.dump(drift, open(f"break/drift_{hypers_as_str}.json", "w"))
    # Plot results for this asymmetry setting
    plot_as_grid(test_errs_global, f"break/test_errs_global_{hypers_as_str}.png")
    plot_as_grid(test_errs_local, f"break/test_errs_local_{hypers_as_str}.png")
    plot_as_grid(drift, f"break/drift_{hypers_as_str}.png")

2026-01-07 13:10:59.115369: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


  0%|          | 0/16 [00:00<?, ?it/s]

2026-01-07 13:11:10.837961: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2026-01-07 13:11:10.838000: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2026-01-07 13:11:10.838071: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2026-01-07 13:11:20.052477: W external/xla/xla/service/gpu/au

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

ValueError: Incorrect number of leaves, expected 10 leaves, but got 20.